In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
batch_size = 32
epochs = 10
img_height = 224
img_width = 224

In [7]:

datagen = ImageDataGenerator(rescale=1/255, validation_split=0.2)

train_generator = datagen.flow_from_directory('animals',
                                               target_size=(img_height, img_width), 
                                               batch_size=batch_size, 
                                               class_mode='categorical', 
                                               subset='training')

validation_generator = datagen.flow_from_directory('animals', 
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    subset='validation')

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [9]:
# Criando CNN do zero com suas layers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', 
                        input_shape=(img_height, img_width, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 52, 52, 64)        36928     
                                                                 
 flatten_1 (Flatten)         (None, 173056)           

In [11]:
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.RMSprop(learning_rate=1e-4), 
              metrics=['acc'])

In [9]:
# Teste config GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17325860817034424026
xla_global_id: -1
]


In [14]:
model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

Epoch 1/10
75/75 [==============================] - 85s 1s/step - loss: 0.8118 - acc: 0.5942 - val_loss: 0.7920 - val_acc: 0.5900
Epoch 2/10
75/75 [==============================] - 85s 1s/step - loss: 0.7028 - acc: 0.6504 - val_loss: 0.7553 - val_acc: 0.5950
Epoch 3/10
75/75 [==============================] - 86s 1s/step - loss: 0.6468 - acc: 0.6950 - val_loss: 0.7105 - val_acc: 0.6300
Epoch 4/10
75/75 [==============================] - 88s 1s/step - loss: 0.5863 - acc: 0.7329 - val_loss: 0.6779 - val_acc: 0.6517
Epoch 5/10
75/75 [==============================] - 78s 1s/step - loss: 0.5298 - acc: 0.7633 - val_loss: 0.7037 - val_acc: 0.6183
Epoch 6/10
75/75 [==============================] - 78s 1s/step - loss: 0.4843 - acc: 0.7900 - val_loss: 0.6370 - val_acc: 0.6900
Epoch 7/10
75/75 [==============================] - 77s 1s/step - loss: 0.4356 - acc: 0.8196 - val_loss: 0.6509 - val_acc: 0.6700
Epoch 8/10
75/75 [==============================] - 85s 1s/step - loss: 0.3841 - acc: 0.85

In [17]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict(validation_generator)
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['cats', 'dogs', 'panda']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))



19/19 [==============================] - 5s 210ms/step
Confusion Matrix
[[72 71 57]
 [67 68 65]
 [55 78 67]]
Classification Report
              precision    recall  f1-score   support

        cats       0.37      0.36      0.37       200
        dogs       0.31      0.34      0.33       200
       panda       0.35      0.34      0.34       200

    accuracy                           0.34       600
   macro avg       0.35      0.34      0.35       600
weighted avg       0.35      0.34      0.35       600

